# Laborator 0: Pattern matching - probleme si solutii

## Curs: Limbaje formale si automate
## Autori: Muraru George si Pavel Cristian

# Partea 0: Recap lab 0

In laboratorul 0 am discutat despre o serie de notiuni introductive pentru limbajul Python. Inainte de inceperea efectiva a laboratorului 1, ar trebui sa puteti raspunde la urmatoarele intrebari

1. Cum declar si definesc o functie?
2. Cum pot sa declar, sa parcurg si sa inserez intr-o lista?
3. Cum declar un string si cum pot sa adaug caractere la acesta?
4. Ce este un dictionar?

# Partea 1: Problema

In laboratorul 1 vom porni de la problema gasirii unui substring intr-un string - pattern matching. Aceasta este prima problema care apare atunci cand dorim sa parsam un text si, pentru a putea ajunge la solutii mai complexe de analiza a limbajelor, trebuie parcurse mai multe solutii cu avantajele si dezavantajele fiecareia.

Asadar astazi ne propunem sa rezolvam urmatoarea problema:

### Enuntul problemei:

  Scrieti o functie care primeste 2 stringuri **text** si **pattern**, si verifica daca **pattern** este un substring al lui **text**.
    
  Exemplu:
    text="scaunmasamasina"
    pattern="masina"
        
    Output: true    
        
# Partea 2: Solutii

## Partea 2.1: Solutia bruta

Trebuie sa pornim de undeva. Prima solutie este cea bruta pe care va invit sa o implementati mai jos:

In [1]:
# Prima solutie

def is_substring_brute_force(text, pattern):
    """
      :param text: Stringul in care trebuie cautat.
      :param pattern: Substringul care trebuie cautat.
      :return: True daca pattern este substring a lui text, altfel False
    """
    ## any idea is a good idea
    pass
    

print(is_substring_brute_force("scaunmasamasina", "masina"));

None


In [18]:

def __is_substring_brute_force__(text, pattern):
    for i in range(len(text) - len(pattern) + 1):
        is_substring = True
        for j in range(len(pattern)):
            if text[i + j] != pattern[j]:
                is_substring = False
        if is_substring is True:
            return is_substring    
    return False

text1 = "scaunmasamasina"
pattern1 = "masina"

print("{0} is substring of {1}: {2}" \
      .format(pattern1, text1, __is_substring_brute_force__(text1, pattern1)))

text2 = "anaaremeresipere"
pattern2 = "anamere"
print("{0} is substring of {1}: {2}" \
      .format(pattern2, text2, __is_substring_brute_force__(text2, pattern2)))

masina is substring of scaunmasamasina: True
anamere is substring of anaaremeresipere: False


### Teste

In [0]:
assert (__is_substring_brute_force__("scaunmasamasina", "masina") == True)
assert (__is_substring_brute_force__("aaabbcccddd", "aaacc") == False)
assert (__is_substring_brute_force__("a3b5c3", "b5c") == True)
assert (__is_substring_brute_force__("LFAisCool", "ool") == True)


## Partea 2.2: O imbunatatire

Pornind de la solutia bruta, putem observa urmatoarele lucruri:
  * La un mismatch, continuam parcurgerea pana la finalul stringului **pattern**
  * In caz de mismatch pornim de la pozitia 0 din **pattern** si incercam din nou sa facem match
  * Nu ne folosim deloc de informatia legata de caracterul care a produs mismatch, si locul in care s-a produs

Acum ne putem gandi la o imbunatatire care rezolva aceste probleme.

Prima data, ne dam seama ca, odata intalnit un mismatch, nu mai e nevoie sa continuam cu parcurgerea celor doua stringuri, ci putem sa incercam un nou match.

De asemenea, o intrebare buna ar fi: **Cum am putea sa ne folosim de caracterul si locul in care s-a produs mismatch pentru a imbunatati performanta algoritmului?**

Vom porni de la urmatorul caz:

    text = "acddbcdebacdbqwr"
    pattern = "acdb"
    
Parcurgem algoritmul brut pas cu pas:

    i = 0 => j = 0 => 'a' = 'a'
             j = 1 => 'b' = 'b'
             j = 2 => 'c' = 'c'
             j = 3 => 'd' != 'b'MISMATCH
             ...
    i = 1 => j = 0 => 'b' != 'a' MISMATCH
             ...
    i = 2 => j = 0 => 'c' != 'a' MISMATCH
             ...
    i = 3 => j = 0 => 'd' != 'a' MISMATCH
             ...
    ...

    i = 9 => j = 0 => 'a' = 'a'
             j = 1 => 'c' = 'c'
             j = 2 => 'd' = 'd'
             j = 3 => 'b' = 'b' MATCH
            
Observam ca pentru cazul in care **i** este egal cu 1, 2 si 3 cautarea a fost facuta inutil.

Stim ca in stringul **text** niciun caracter de pe pozitiile 1, 2 si 3 **nu** este 'a', informatie obtinuta cand a fost facut matching pentru i = 0.

Deci avand informatia **La pozitia 3 s-a facut mismatch pe caracterul 'd'** am fi putut sari direct la pozitia 3.

De ce avem nevoie pentru a putea face acest salt?

Uitandu-ne la string-ul **pattern**, vedem ca, in cazul unui mismatch $pattern[j] \neq text[i]$, noi putem continua cu $j = 0$, cu $i$ neschimbat, deoarece daca am ajuns la pozitia j in sirul B, atunci $B[0] = A[i - j], B[1] = A[i - j + 1] ... B[j - 1] = A[i - 1]$. Acum, de ce nu inaintam, ca in algoritmul brut, si nu comparam $B[0]$ cu $A[i - j + 1], A[i - j + 2],...,A[i]$ ? Pentru ca, uitandu-ne la B, vedem ca $B[0]$ nu apare de doua ori in B si $B[0] = A[i - j], B[1] = A[i - j + 1] ... B[j - 1] = A[i - 1]$.

Generalizand, noi avem nevoie de o tabela T care sa ne spuna, daca s-a facut mismatch pe pozita j in B, continuam de la pozitia $T[j]$ in B si inaintam sau nu in A.



In [0]:
def construct_table(pattern):
    ## T e tabelul de care ne spune unde sa ne mutam la un mismatch
    T = [0] * (len(pattern) + 1)
    T[0] = -1
    indx = 0  # indx este indexul caracterului la care trebuie sa ne ducem in B pentru o noua incercare
    for i in range(1, len(b)):
        """
         daca pattern[i] si pattern[indx] sunt egale inseamna ca pentru un
         mismatch la i reactionam la fel ca pentru un mismatch la indx
        """
        if b[i] == b[indx]: 
            T[i] = T[indx]
        else:
            T[i] = indx  # pentru un mismatch la b[i] ne vom duce la indx
            indx = T[indx]
            while indx >= 0 and b[i] != b[indx]: # cautam noua pozitie
                indx = T[indx]
        indx += 1
    T[len(b)] = indx # Remember: T[len(b)] == T[-1]
    return T

def is_substring_improved(a, b):
    T = construct_table(b)
    i = 0
    j = 0
    while i < len(a):
        if b[j] == a[i]:
            j += 1
            i += 1
            if j == len(b):
                return True
        else:
            j = T[j]
            if j < 0:
                j += 1
                i += 1
    return False
                
print(is_substring_improved("scaunmasamasina", "scaun"))
        
    
    

True


## Partea 3: De la tabela la automat

Tabela de mai devreme poate fi vazuta ca un automat. Un automat este o ......